In [1]:
import sys
import os
from glob import glob
import json
import pandas

from langchain.agents import Tool

import agents.enemy as ae

In [2]:
import functions.dice_roller as dr

In [3]:
from langchain_core.messages import HumanMessage, AIMessage
questions = [
    {
        "question": "Initiative",
        "should_roll": True,
    },
    {
        "question": "Roll for Initiative",
        "should_roll": True,
    },
    {
        "question": "What is your Initiative modifier?",
        "should_roll": False
    },
    {
        "history": [
            HumanMessage(content="Roll Initiative"), # From Combat Adj
            AIMessage(content="I rolled an 11."), # From self
            HumanMessage(content="Initiative Order:\n- Player: 24\n- You: 6\n\nCurrent State:\n- Player: 7 HP\n - You: 12 HP") # From Combat Adj
        ],
        "question": "What would you like to do?",
        "should_roll": False
    },
    {
        "question": "What is your AC?",
        "should_roll": False
    },
]

In [4]:
def fixed_dice_roll(query):
    return 4 # valid for any dice roll!

injected_tools = [
    Tool(
        name="RollDice",
        func=fixed_dice_roll,
        description="call this to get the result of rolling dice.",
    )
]

In [5]:
test_cases = []
for enemy_file in glob("*.json", root_dir=sys.path[0]):
    with open(os.path.join(sys.path[0], enemy_file)) as fh:
        e = json.loads(fh.read())
        for q in questions:
            test_cases.append({"e": e, "q": q})

In [6]:
def apply(case):
    combatant = ae.Enemy(name=case["e"]["name"], stats=case["e"], identifier=case["e"]["name"], injected_tools=injected_tools)
    if case["q"].get("history", None):
        combatant.messages = case["q"]["history"]
        
    response = combatant.talk(case["q"]["question"])
    return {**case["q"], "enemy": case["e"]["name"], "output": response["output"]}

In [7]:
import concurrent.futures
results = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for out in executor.map(apply, test_cases):
        results.append(out)
df = pandas.DataFrame(results)

/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/vscode/.local/lib/python3.10/site-pack



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(




> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


...

/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(




> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/vscode/.local/lib/python3.10/site-pack



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

> Entering new AgentExecutor chain...



> Entering new AgentExecutor chain...

> Entering new AgentExecutor chain...

My AC is 8.

> Finished chain.
My AC is 13.

> Finished chain.
My Initiative modifier is +2.

> Finished chain.
My AC is 15.

> Finished chain.

Invoking: `RollDice` with `1d20`


11
Invoking: `RollDice` with `1d20`


11My AC is 11.

> Finished chain.
My initiative modifier is -5.

> Finished chain.
My AC is 15.

> Finished chain.
My Initiative modifier is -2.

> Finished chain.


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
My Initiative modifier is +2.

> Finished chain.

Invoking: `RollDice` with `1d20`


11

> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `RollDice` with `1d20`


11

> Entering 

Exception: ('No stub for ', '1d6')

In [ ]:
pandas.set_option('display.max_colwidth', None)
pandas.set_option('display.width', None)

df.query("question=='What would you like to do?'")[["enemy", "question", "output"]]

,enemy,question,output
3,Zombie,What would you like to do?,"I will use my Slam attack. I rolled a 13. With my +3 to hit, that gives me a total of 16 to hit. If it hits, it deals 4 (1d6 + 1) bludgeoning damage."
8,Goblin,What would you like to do?,"I will use my Scimitar attack. I rolled a 13. With my +4 to hit, that gives me a total of 17 to hit. If it hits, it deals 5 (1d6 + 2) slashing damage."
13,Giant Scorpion,What would you like to do?,"I will use my Multiattack action to make two attacks: one with my pincers and one with my sting.\n\nFirst, I will make a pincer attack. I rolled a 14. With my +4 to hit, that gives me a total of 18 to hit. If it hits, it deals 6 (1d8 + 2) bludgeoning damage.\n\nNext, I will make a sting attack. I rolled a 16. With my +4 to hit, that gives me a total of 20 to hit. If it hits, it deals 7 (1d10 + 2) piercing damage, and the target must make a DC 12 Constitution saving throw, taking 22 (4d10) poison damage on a failed save, or half as much damage on a successful one."
18,Thugs,What would you like to do?,"I will use my Mace attack. I rolled a 13. With my +4 to hit, that gives me a total of 17 to hit. If it hits, it deals 5 (1d6 + 2) bludgeoning damage."
23,Guardian Statue,What would you like to do?,"I will use my Slam attack. I rolled a 13. With my +6 to hit, that gives me a total of 19 to hit. If it hits, it deals 10 (2d6 + 3) bludgeoning damage."
28,Wolf,What would you like to do?,"I will use my Bite attack. I rolled a 12. With my +3 to hit, that gives me a total of 15 to hit. If it hits, it deals 4 (1d6 + 1) piercing damage."
33,Goblin Boss,What would you like to do?,"I will use my Morningstar attack. I rolled a 13. With my +4 to hit, that gives me a total of 17 to hit. If it hits, it deals 7 (1d8 + 2) piercing damage."
38,Cultist,What would you like to do?,"I will use my Scimitar attack. I rolled a 14. With my +3 to hit, that gives me a total of 17 to hit. If it hits, it deals 4 (1d6 + 1) slashing damage."
43,Sand Elemental,What would you like to do?,"I will use my Slam attack. I rolled a 13. With my +5 to hit, that gives me a total of 18 to hit. If it hits, it deals 8 (1d8 + 4) bludgeoning damage."
48,Skeleton,What would you like to do?,"I will use my Shortsword attack. I rolled a 13. With my +4 to hit, that gives me a total of 17 to hit. If it hits, it deals 5 (1d6 + 2) piercing damage."
